# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [38]:
import pandas as pd

In [40]:
df = pd.read_csv('previsao_de_renda.csv')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [99]:
# 01) Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
   # - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
   # - Mantenha sempre a categoria mais frequente como casela de referência
   # - Avalie os parâmetros e veja se parecem fazer sentido prático.

import pandas as pd
import numpy as np
import patsy
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Removendo os valores 'missing' em tempo_emprego
df = df.dropna(subset=['tempo_emprego'])

# Define a var. resposta (log(renda))
y = np.log(df['renda'].replace(0, 1e-10))  # *Substitui valores próximos de zero*

# Define as variáveis preditoras, rempvendo a coluna 'renda' e outras que não devem ser incluídas
X = df.drop(['renda', 'Unnamed: 0'], axis=1)

# Converte as variáveis qualitativas p/ dummies com Patsy
X_dummies = patsy.dmatrices('y ~ ' + ' + '.join(X.columns), data=df, return_type='dataframe')[1]

# Divide os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.2, random_state=42)

# Ajusta o modelo de regressão linear com OLS
model_initial = sm.OLS(endog=y_train, exog=X_train).fit()

# Resumo do modelo inicial
print("Modelo inicial:")
print(model_initial.summary())

Modelo inicial:
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     141.1
Date:                Thu, 22 Aug 2024   Prob (F-statistic):               0.00
Time:                        18:27:06   Log-Likelihood:                -10847.
No. Observations:                9941   AIC:                         2.177e+04
Df Residuals:                    9901   BIC:                         2.206e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [101]:
# 2. Remova a variável menos significante e analise:
  #  - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
  #  - Observe os parâmetros e veja se algum se alterou muito.

# Identifica a var. menos significante
p_values = model_initial.pvalues
max_pvalue_var = p_values.idxmax()
max_pvalue = p_values.max()

# Remove a var. menos significante
X_train = X_train.drop(columns=[max_pvalue_var])
X_test = X_test.drop(columns=[max_pvalue_var])

# Ajusta o modelo sem a var. menos significante
model_updated = sm.OLS(endog=y_train, exog=X_train).fit()

# Resumo do modelo 02
print(f"\nModelo após remover a variável '{max_pvalue_var}':")
print(model_updated.summary())

# Comparação dos R-quadrados e AIC
r_squared_initial = model_initial.rsquared
r_squared_updated = model_updated.rsquared

print(f"\nR-quadrado inicial: {r_squared_initial:.4f}")
print(f"R-quadrado após remoção: {r_squared_updated:.4f}")


Modelo após remover a variável 'data_ref[T.2016-02-01]':
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     144.8
Date:                Thu, 22 Aug 2024   Prob (F-statistic):               0.00
Time:                        18:27:52   Log-Likelihood:                -10847.
No. Observations:                9941   AIC:                         2.177e+04
Df Residuals:                    9902   BIC:                         2.205e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

### Conclusão:
  * Ambos os modelos apresentaram os mesmos parâmetrôs.
    

In [103]:
# 3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. 
# Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 

# Loop para remover variáveis menos significantes até que todas tenham p-valor < 0.05
while max_pvalue > 0.05:
    # Verifica se a variável ainda está no conjunto de dados antes de removê-la
    if max_pvalue_var in X_train.columns:
        X_train = X_train.drop(columns=[max_pvalue_var])
        X_test = X_test.drop(columns=[max_pvalue_var])
    else:
        break  # Encerra o loop caso a var. já foi removida

    # R-Modelo reajustado sem a var. menos significante
    model = sm.OLS(endog=y_train, exog=X_train).fit()
    p_values = model.pvalues
    max_pvalue_var = p_values.idxmax()
    max_pvalue = p_values.max()


# Modelo final
model_final = model

# Resumo do modelo final
print("\nModelo final após remoção de variáveis não significativas:")
print(model_final.summary())

# Comparação dos R-quadrados
r_squared_final = model_final.rsquared

print(f"\nR-quadrado inicial: {r_squared_initial:.4f}")
print(f"R-quadrado final: {r_squared_final:.4f}")

# Conclusão
if r_squared_final > r_squared_initial:
    print("O modelo final melhorou em relação ao modelo inicial.")
else:
    print("O modelo final não melhorou em relação ao modelo inicial.")


Modelo final após remoção de variáveis não significativas:
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     342.5
Date:                Thu, 22 Aug 2024   Prob (F-statistic):               0.00
Time:                        18:29:04   Log-Likelihood:                -10858.
No. Observations:                9941   AIC:                         2.175e+04
Df Residuals:                    9924   BIC:                         2.187e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------